<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022%20/%20Image_07%20/class_007_image_processing_smart_trash_bin_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 193MB/s]


In [4]:
FILE='trash_nov_22_2018.zip'

In [5]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [6]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [7]:
Img_Size = 224
ref = 'jpg'
notref = 'met'

In [8]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    img = cv2.imread(name)
    resized = cv2.resize(img, (Img_Size,Img_Size))
    X.append(resized)
m = len(Y)
print(n,m)

2527 2527


In [9]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [10]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [11]:
df=pd.DataFrame(label,columns=['target'])

In [12]:
y = pd.get_dummies(df['target'])

In [13]:
for i in range(6): 
  n= df[(df["target"] == i)].shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [14]:
#label

In [15]:
n =len(img_name)
data = np.empty((n, 224, 224, 3))

In [16]:
# https://pythontutorials.eu/deep-learning/transfer-learning/

In [17]:
DATASET_SIZE = len(img_name)
Ind = range(DATASET_SIZE)
Ind_train, Ind_test= train_test_split(Ind,test_size=0.20,stratify=label, shuffle=True, random_state=3)
X = np.array(X)
X_train=X[Ind_train]
X_test = X[Ind_test]
y_train=y.iloc[Ind_train,:]
y_test = y.iloc[Ind_test,:]

In [18]:
from keras.applications.mobilenet_v2 import preprocess_input
from skimage.transform import resize
i = 0
for im in X:
    im = preprocess_input(im)
    im = resize(im, output_shape=(224, 224))
    data[i] = im
    i += 1

In [19]:
n

2527

In [20]:
n =len(img_name)
m = len(tipos)

'''
labels_tf = np.empty(n, dtype=int)
for idx, yi in enumerate(label):
  labels_tf[idx] = yi
'''
idx = 0
labels_tf = np.empty([n,m], dtype=int)
for yi in y.values:
  labels_tf[idx] = yi
  idx = idx + 1



In [21]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(weights='imagenet')

In [22]:
from keras.applications.mobilenet_v2 import decode_predictions

predictions = model.predict(data)
k = 0
for decoded_prediction in decode_predictions(predictions, top=1):
  
  for name, desc, score in decoded_prediction:
      if(k % 200 ==0):
        print('- {} ({:.2f}%%) {}'.format(desc, 100 * score,img_name[k]))
  k = k +1

79/79 [==============================] - 6s 38ms/step
- envelope (27.77%%) cardboard1.jpg
- mailbag (27.79%%) cardboard201.jpg
- velvet (10.09%%) cardboard401.jpg
- ballpoint (13.35%%) glass198.jpg
- spotlight (96.14%%) glass398.jpg
- water_bottle (24.05%%) metal97.jpg
- ashcan (71.91%%) metal297.jpg
- syringe (20.62%%) paper87.jpg
- shopping_cart (10.26%%) paper287.jpg
- hand-held_computer (23.41%%) paper487.jpg
- water_bottle (18.48%%) plastic93.jpg
- water_bottle (60.90%%) plastic293.jpg
- plastic_bag (69.38%%) trash11.jpg


In [23]:
from keras.layers import Dense
D = len(tipos)
model_output = Dense(D, activation='softmax')

In [24]:
model_output = model_output(model.layers[-2].output)

In [25]:
from keras import Model

model_input = model.input
model_model = Model(inputs=model_input, outputs=model_output)

In [26]:
for layer in model_model.layers[:-1]:
    layer.trainable = False

In [27]:
model_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
# model_model.fit(x=data, y=labels_tf, epochs=20, verbose=2) 
model_model.fit(x=X_train, y=y_train, epochs=50, verbose=2) 

Epoch 1/50
64/64 - 5s - loss: 1.3530 - accuracy: 0.4800 - 5s/epoch - 82ms/step
Epoch 2/50
64/64 - 2s - loss: 1.0352 - accuracy: 0.6170 - 2s/epoch - 36ms/step
Epoch 3/50
64/64 - 2s - loss: 0.9292 - accuracy: 0.6650 - 2s/epoch - 36ms/step
Epoch 4/50
64/64 - 2s - loss: 0.8648 - accuracy: 0.6917 - 2s/epoch - 36ms/step
Epoch 5/50
64/64 - 2s - loss: 0.8301 - accuracy: 0.7115 - 2s/epoch - 35ms/step
Epoch 6/50
64/64 - 2s - loss: 0.7781 - accuracy: 0.7279 - 2s/epoch - 35ms/step
Epoch 7/50
64/64 - 2s - loss: 0.7428 - accuracy: 0.7462 - 2s/epoch - 36ms/step
Epoch 8/50
64/64 - 2s - loss: 0.7106 - accuracy: 0.7615 - 2s/epoch - 36ms/step
Epoch 9/50
64/64 - 2s - loss: 0.6837 - accuracy: 0.7660 - 2s/epoch - 36ms/step
Epoch 10/50
64/64 - 2s - loss: 0.6657 - accuracy: 0.7783 - 2s/epoch - 35ms/step
Epoch 11/50
64/64 - 2s - loss: 0.6416 - accuracy: 0.7912 - 2s/epoch - 36ms/step
Epoch 12/50
64/64 - 2s - loss: 0.6260 - accuracy: 0.7872 - 2s/epoch - 36ms/step
Epoch 13/50
64/64 - 2s - loss: 0.6103 - accuracy:

In [29]:
def retorna(y):
  resp = []
  for yi in y:
    vmax =max(yi)
    for idx, x in enumerate(yi):
      if(x == vmax):
        resp.append(idx)
  return resp

In [30]:
test_loss, test_acc = model_model.evaluate(X_test,  y_test, verbose=2)

16/16 - 2s - loss: 0.7733 - accuracy: 0.7213 - 2s/epoch - 123ms/step


In [31]:
predict=model_model.predict(X_test)

16/16 [==============================] - 1s 36ms/step


In [32]:
predict[0].shape

(6,)

In [33]:
y_calc = retorna(predict)
y_test = retorna(np.array(y_test))

In [34]:
confusion_matrix(y_test, y_calc)

array([[ 68,   2,   1,   7,   3,   0],
       [  0,  70,  15,   4,  11,   0],
       [  0,   7,  61,   2,  10,   2],
       [  7,   1,   3, 105,   1,   2],
       [  1,  20,   8,  13,  54,   1],
       [  1,   5,   0,   7,   7,   7]])

In [35]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       0.88      0.84      0.86        81
       glass       0.67      0.70      0.68       100
       metal       0.69      0.74      0.72        82
       paper       0.76      0.88      0.82       119
     plastic       0.63      0.56      0.59        97
       trash       0.58      0.26      0.36        27

    accuracy                           0.72       506
   macro avg       0.70      0.66      0.67       506
weighted avg       0.72      0.72      0.71       506

